# Objective
- In this python workbook, I am trying to calculate the Spot prices of options.
- I will need that information later when I will optimize the Heston model

In [1]:
import numpy as np
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from matplotlib import cm

from scipy import interpolate

### Extracting data from Market datafile

In [2]:
data = pd.read_excel('data_apple.xlsx')
data.head()

,Option_type,Maturity_days,Strike,Ticker,Bid,Ask,Last,IVM,Volm
0,Call,25,170.0,AAPL 8/17/18 C170,21.150000,21.400000,20.65,30.762677,7
1,Call,25,172.5,AAPL 8/17/18 C172.5,18.649994,19.049988,0.00,29.173643,0
2,Call,25,175.0,AAPL 8/17/18 C175,16.449997,16.649994,16.50,28.306871,19
3,Call,25,177.5,AAPL 8/17/18 C177.5,14.100000,14.400000,0.00,26.967507,0
4,Call,25,180.0,AAPL 8/17/18 C180,12.050000,12.150000,12.10,26.321682,1129


In [3]:
# Drop column
data = data.drop(columns=['Ticker','Volm','Bid','Ask'])
# Rename column 
data = data.rename(columns={'Last': 'Market_price'})

# Implied Volatility (absolute value from percentage)
data['IVM'] = data['IVM']/100

# Annualized days (Number of trading days in a year = 252)
data['Maturity_days'] = data['Maturity_days']

data.head()

,Option_type,Maturity_days,Strike,Market_price,IVM
0,Call,25,170.0,20.65,0.307627
1,Call,25,172.5,0.00,0.291736
2,Call,25,175.0,16.50,0.283069
3,Call,25,177.5,0.00,0.269675
4,Call,25,180.0,12.10,0.263217


Calculate $S_0$ using Put-Call parity

For same strike price (K)

$S_0=C-P+Ke^{-rT}$

In [4]:
import pandas as pd
import numpy as np

# Constants
r = 0.05  # risk-free rate

# Create a new column for S0 (spot price)
data['S0'] = np.nan  # Initialize S0 column with NaN

# Loop through unique combinations of Maturity_days and Strike
for _, group in data.groupby(['Maturity_days', 'Strike']):
    T = group['Maturity_days'].iloc[0] / 252.0  # Convert maturity to years assuming 252 trading days in a year
    strike = group['Strike'].iloc[0]
    
    # Extract call and put prices
    call_price = group[group['Option_type'] == 'Call']['Market_price'].values
    put_price = group[group['Option_type'] == 'Put']['Market_price'].values
    
    if len(call_price) > 0 and len(put_price) > 0:  # Ensure both call and put prices exist
        call_price = call_price[0]
        put_price = put_price[0]

        # Calculate S0 using put-call parity
        S0 = (call_price - put_price + strike * np.exp(-r * T))
        
        # Update the DataFrame with the calculated S0 for the current group
        data.loc[group.index, 'S0'] = S0
    else:
        print(f"Warning: Missing call or put price for Maturity_days={T*252}, Strike={strike}")

# Display the updated DataFrame with the S0 column
print(data)


    Option_type  Maturity_days  Strike  Market_price       IVM          S0
0          Call             25   170.0     20.650000  0.307627  189.368834
1          Call             25   172.5      0.000000  0.291736  171.066464
2          Call             25   175.0     16.500000  0.283069  189.844094
3          Call             25   177.5      0.000000  0.269675  175.541724
4          Call             25   180.0     12.100000  0.263217  189.799354
..          ...            ...     ...           ...       ...         ...
301         Put            697   210.0     33.480000  0.221666  168.517017
302         Put            697   215.0     35.500000  0.222254  169.071230
303         Put            697   220.0     38.099998  0.218975  167.985446
304         Put            697   225.0     41.950001  0.219277  167.589659
305         Put            697   230.0     45.500000  0.220995  166.903874

[306 rows x 6 columns]


In [5]:
# Create a filtered DataFrame for Call options, storing relevant columns
df = data[data['Option_type'] == 'Call'].copy()  # Filter only 'Call' options
df.head()


,Option_type,Maturity_days,Strike,Market_price,IVM,S0
0,Call,25,170.0,20.65,0.307627,189.368834
1,Call,25,172.5,0.00,0.291736,171.066464
2,Call,25,175.0,16.50,0.283069,189.844094
3,Call,25,177.5,0.00,0.269675,175.541724
4,Call,25,180.0,12.10,0.263217,189.799354


In [6]:
nan_count = df['S0'].isna().sum()
#print(f"Number of NaN values in 'S0': {nan_count}")

# Drop rows where 'S0' is NaN
df = df.dropna(subset=['S0'])
df.head()

,Option_type,Maturity_days,Strike,Market_price,IVM,S0
0,Call,25,170.0,20.65,0.307627,189.368834
1,Call,25,172.5,0.00,0.291736,171.066464
2,Call,25,175.0,16.50,0.283069,189.844094
3,Call,25,177.5,0.00,0.269675,175.541724
4,Call,25,180.0,12.10,0.263217,189.799354


### Optimize to get implied volatility

In [7]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import brentq  # More stable for root-finding

# Black-Scholes call pricing model
def black_scholes_call(S, K, T, r, q, sigma):
    """Calculate the Black-Scholes call option price"""
    
    d1 = (np.log(S/K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)

# Objective function to calculate implied volatility
def implied_volatility_for_row(row, r=0.0245, q=0.005):
    """Calculate implied volatility for a single option using brentq"""
    
    # Convert maturity days to years (252 days of trading) 
    T_years = row['Maturity_days'] / 252.0  # Assuming 'Maturity_days' is in days
    
    # Define the objective function for root-finding
    def objective(sigma):
        model_price = black_scholes_call(row['S0'], row['Strike'], T_years, r, q, sigma)
        return model_price - row['Market_price']  # We want the model price to match market price

    # Use brentq (root-finding) between reasonable bounds
    try:
        sigma = brentq(objective, 1e-6, 1.0)  # Volatility range between 0 and 1
        return round(sigma, 2)
    except ValueError:
        # If the optimization fails, return NaN
        return np.nan

# Add implied volatility to the DataFrame
def calculate_implied_volatility(df):
    # Calculate implied volatility using the above method
    df['Implied_volatility'] = df.apply(implied_volatility_for_row, axis=1)
    return df

# Example usage with your DataFrame
df_with_volatility = calculate_implied_volatility(df)

# Round all numerical columns to 2 decimal places
df_with_volatility = df_with_volatility.round(2)
df_with_volatility = df_with_volatility.rename(columns={'Implied_volatility':'vol'})

# Print the DataFrame with both 'IVM' and 'Implied_volatility' columns
print(df_with_volatility)


    Option_type  Maturity_days  Strike  Market_price   IVM      S0   vol
0          Call             25   170.0         20.65  0.31  189.37  0.29
1          Call             25   172.5          0.00  0.29  171.07  0.00
2          Call             25   175.0         16.50  0.28  189.84  0.27
3          Call             25   177.5          0.00  0.27  175.54  0.00
4          Call             25   180.0         12.10  0.26  189.80  0.24
..          ...            ...     ...           ...   ...     ...   ...
284        Call            697   210.0         19.12  0.22  168.52  0.27
285        Call            697   215.0         17.34  0.23  169.07  0.26
286        Call            697   220.0         14.50  0.22  167.99  0.25
287        Call            697   225.0         13.60  0.22  167.59  0.25
288        Call            697   230.0         12.11  0.22  166.90  0.25

[153 rows x 7 columns]


I will compute the difference between the implied volatility I calculated (`vol`) and the one already provided (`IVM`). If the absolute difference exceeds 0.1, I will drop that row, as this could indicate potential errors in the values for the call or put prices, or the strike price.


In [8]:
# Calculate the vol_diff (difference between IVM and calculated implied volatility)
df_with_volatility['vol_diff'] = df_with_volatility['vol'] - df_with_volatility['IVM']

# Drop rows where vol_diff values are greater than 0.1 (absolute value)
df_with_volatility = df_with_volatility[abs(df_with_volatility['vol_diff']) <= 0.1]
print(df_with_volatility.head())
print('Dataframe shape: ',df_with_volatility.shape)

  Option_type  Maturity_days  Strike  Market_price   IVM      S0   vol  \
0        Call             25   170.0         20.65  0.31  189.37  0.29   
2        Call             25   175.0         16.50  0.28  189.84  0.27   
4        Call             25   180.0         12.10  0.26  189.80  0.24   
5        Call             25   182.5          9.65  0.25  189.19  0.23   
6        Call             25   185.0          8.15  0.25  189.57  0.23   

   vol_diff  
0     -0.02  
2     -0.01  
4     -0.02  
5     -0.02  
6     -0.02  
Dataframe shape:  (150, 8)


In [9]:
# Saving it as DataFrame
#df_with_volatility.to_csv('apple_data_with_volatility.csv', index=False)
